In [1]:
def grep(pattern):
    print("start grep for", pattern)
    i = 0
    while True:
        s = yield i
        i += 1
        if pattern in s:
            print("found!", s)
        else:
            print("no %s in %s" % (pattern, s))

In [2]:
g = grep("python")

In [3]:
g

<generator object grep at 0x1041995b0>

In [4]:
next(g)

start grep for python


0

In [5]:
g.send("abc")

no python in abc


1

In [6]:
g.send("python3.10")

found! python3.10


2

In [7]:
g.throw(ValueError(5))

ValueError: 5

In [8]:
g.send("dwd")

StopIteration: 

In [9]:
import asyncio

In [10]:


@asyncio.coroutine
def count():
    res = yield from [1,2,3]
    for i in [1,2,3]:
        yield i
    return res

/var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_95399/2581609868.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def count():


In [12]:


async def async_count(n):
    await 

In [14]:
await async_count(10)

RuntimeError: Task got bad yield: 10

In [19]:
async def countdown(n):
    for i in range(n):
        print("countdown", n, i)
        await asyncio.sleep(1)

In [20]:
await countdown(5)

countdown 5 0
countdown 5 1
countdown 5 2
countdown 5 3
countdown 5 4


In [21]:
await countdown(5)
await countdown(7)

countdown 5 0
countdown 5 1
countdown 5 2
countdown 5 3
countdown 5 4
countdown 7 0
countdown 7 1
countdown 7 2
countdown 7 3
countdown 7 4
countdown 7 5
countdown 7 6


In [23]:
async def fn():
    await countdown(5)
    await countdown(7)

In [24]:
await fn()

countdown 5 0
countdown 5 1
countdown 5 2
countdown 5 3
countdown 5 4
countdown 7 0
countdown 7 1
countdown 7 2
countdown 7 3
countdown 7 4
countdown 7 5
countdown 7 6


In [41]:
async def fib(n):
    await asyncio.sleep(0)
    return await fib(n - 1) + await fib(n - 2)


async def coro():
    print("coro")
    
    import time
#     time.sleep(5)

    fib(50)

    for i in range(100):
        time.sleep(5 / 100)
        await asyncio.sleep(0)
    
    return 999


async def fn():
    task = asyncio.create_task(countdown(3))
    task1 = asyncio.create_task(countdown(2))

    print("task created")

    res = await coro()
    print("coro res", res)

    await countdown(5)
    await countdown(1)

    print("countdown finished")
#     await task

In [42]:
await fn()

task created
coro
countdown 3 0
countdown 2 0
countdown 3 1
countdown 2 1
countdown 3 2
coro res 999
countdown 5 0
countdown 5 1
countdown 5 2
countdown 5 3
countdown 5 4
countdown 1 0
countdown finished


In [46]:
async def countdown(n):
    for i in range(n):
        yield i
        print('yield', i)
        await asyncio.sleep(1)

async for i in countdown(5):
    print(i)

0
yield 0
1
yield 1
2
yield 2
3
yield 3
4
yield 4


In [52]:
async def async_sleep(t):
    print("sleep", t)
    await asyncio.sleep(t)
    print("end sleep", t)
    return t * 10

In [53]:
import time

In [54]:
t1 = time.perf_counter()

await async_sleep(1)
await async_sleep(2)
await async_sleep(3)

t2 = time.perf_counter()
print(t2 - t1)

sleep 1
end sleep 1
sleep 2
end sleep 2
sleep 3
end sleep 3
6.013157175970264


In [55]:
t1 = time.perf_counter()

tasks = [
    asyncio.create_task(async_sleep(1)),
    asyncio.create_task(async_sleep(2)),
    asyncio.create_task(async_sleep(3)),
]
res = asyncio.gather(*tasks)
await res

t2 = time.perf_counter()
print(t2 - t1)

sleep 1
sleep 2
sleep 3
end sleep 1
end sleep 2
end sleep 3
3.0012569049722515


In [60]:
t1 = time.perf_counter()

tasks = [
    asyncio.create_task(async_sleep(3)),
    asyncio.create_task(async_sleep(2)),
    asyncio.create_task(async_sleep(1)),
]
for t in asyncio.as_completed(tasks):
    res = await t
    print(res)

t2 = time.perf_counter()
print(t2 - t1)

sleep 3
sleep 2
sleep 1
end sleep 1
10
end sleep 2
20
end sleep 3
30
3.0042754290043376


In [143]:
!pip install aiohttp

In [63]:
import aiohttp

In [64]:
URL = "https://docs.python.org/3/library/asyncio-task.html"

In [98]:
async def fetch(url, session, lock):
    async with lock:
        async with session.get(url) as resp:
            data = await resp.read()


async def crawl(urls):
    lock = asyncio.Lock()
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            tasks.append(
                asyncio.create_task(
                    fetch(url, session, lock)
                )
            )
        await asyncio.gather(*tasks)

In [105]:
URLS = [URL] * 20

t1 = time.perf_counter()

await crawl(URLS)

t2 = time.perf_counter()
print(t2 - t1)

1.4196976349921897


In [137]:
async def fetch(name, session, q):
    while True:
        url = await q.get()
        try:
            async with session.get(url) as resp:
                data = await resp.read()
                print("name", name, len(data))
        finally:
            q.task_done()


async def crawl(urls, threads=5):
    q = asyncio.Queue()
    for url in urls:
        await q.put(url)

    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(fetch(f"coro_{i}", session, q))
            for i in range(threads)
        ]
        #await asyncio.gather(*tasks)

        await q.join()
    
    for t in tasks:
        t.cancel()

In [141]:
t1 = time.perf_counter()

URLS = [URL] * 20

await crawl(URLS, 10)

t2 = time.perf_counter()
print(t2 - t1)

name coro_0 106122
name coro_4 106122
name coro_5 106122
name coro_7 106122
name coro_1 106122
name coro_3 106122
name coro_2 106122
name coro_6 106122
name coro_8 106122
name coro_9 106122
name coro_0 106122
name coro_4 106122
name coro_5 106122
name coro_7 106122
name coro_3 106122
name coro_2 106122
name coro_6 106122
name coro_8 106122
name coro_1 106122
name coro_9 106122
0.22238614602247253
